Varational Structure Quantum Generative Adversarial Networks
-----------------------
Penn Ave Fish company

This notebook is a work-in-progress, and does not reflect the final state of this project.

Based on *Quantum Generative Adversarial Networks with Cirq + TensorFlow* at <https://pennylane.ai/qml/demos/tutorial_QGAN.html>


Using Cirq + TensorFlow
-----------------------
PennyLane allows us to mix and match quantum devices and classical machine
learning software. For this demo, we will link together
Google's `Cirq <https://cirq.readthedocs.io/en/stable/>`_ and `TensorFlow <https://www.tensorflow.org/>`_ libraries.

We begin by importing PennyLane, NumPy, and TensorFlow.



In [7]:
import pennylane as qml
import numpy as np
import tensorflow as tf
import remote_cirq

We also declare a 3-qubit simulator device running in Cirq.



In [8]:
# API_KEY = '[REDACTED]'
# sim = remote_cirq.RemoteSimulator(API_KEY)
# dev = qml.device('cirq.simulator', wires=26, simulator=sim, analytic=False)
dev = qml.device('cirq.simulator', wires=3)

Generator and Discriminator
---------------------------

In classical GANs, the starting point is to draw samples either from
some "real data" distribution, or from the generator, and feed them to
the discriminator. In this QGAN example, we will use a quantum circuit
to generate the real data.

For this simple example, our real data will be a qubit that has been
rotated (from the starting state $\left|0\right\rangle$) to some
arbitrary, but fixed, state.



In [9]:
def real(angles, **kwargs):
    qml.Hadamard(wires=0)
    qml.Rot(*angles, wires=0)

For the generator and discriminator, we will choose the same basic
circuit structure, but acting on different wires.

Both the real data circuit and the generator will output on wire 0,
which will be connected as an input to the discriminator. Wire 1 is
provided as a workspace for the generator, while the discriminator’s
output will be on wire 2.



In [10]:
def generator(w, **kwargs):
    qml.Hadamard(wires=0)
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=1)
    qml.RY(w[2], wires=0)
    qml.RY(w[3], wires=1)
    qml.RZ(w[4], wires=0)
    qml.RZ(w[5], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RX(w[6], wires=0)
    qml.RY(w[7], wires=0)
    qml.RZ(w[8], wires=0)


def discriminator(w):
    qml.Hadamard(wires=0)
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=2)
    qml.RY(w[2], wires=0)
    qml.RY(w[3], wires=2)
    qml.RZ(w[4], wires=0)
    qml.RZ(w[5], wires=2)
    qml.CNOT(wires=[0, 2])
    qml.RX(w[6], wires=2)
    qml.RY(w[7], wires=2)
    qml.RZ(w[8], wires=2)

We create two QNodes. One where the real data source is wired up to the
discriminator, and one where the generator is connected to the
discriminator. In order to pass TensorFlow Variables into the quantum
circuits, we specify the ``"tf"`` interface.



In [11]:
@qml.qnode(dev, interface="tf")
def real_disc_circuit(phi, theta, omega, disc_weights):
    real([phi, theta, omega])
    discriminator(disc_weights)
    return qml.expval(qml.PauliZ(2))


@qml.qnode(dev, interface="tf")
def gen_disc_circuit(gen_weights, disc_weights):
    generator(gen_weights)
    discriminator(disc_weights)
    return qml.expval(qml.PauliZ(2))

QGAN cost functions
-------------------

There are two cost functions of interest, corresponding to the two
stages of QGAN training. These cost functions are built from two pieces:
the first piece is the probability that the discriminator correctly
classifies real data as real. The second piece is the probability that the
discriminator classifies fake data (i.e., a state prepared by the
generator) as real.

The discriminator is trained to maximize the probability of
correctly classifying real data, while minimizing the probability of
mistakenly classifying fake data.

\begin{align}Cost_D = \mathrm{Pr}(real|\mathrm{fake}) - \mathrm{Pr}(real|\mathrm{real})\end{align}

The generator is trained to maximize the probability that the
discriminator accepts fake data as real.

\begin{align}Cost_G = - \mathrm{Pr}(real|\mathrm{fake})\end{align}




In [12]:
def prob_real_true(disc_weights):
    true_disc_output = real_disc_circuit(phi, theta, omega, disc_weights)
    # convert to probability
    prob_real_true = (true_disc_output + 1) / 2
    return prob_real_true


def prob_fake_true(gen_weights, disc_weights):
    fake_disc_output = gen_disc_circuit(gen_weights, disc_weights)
    # convert to probability
    prob_fake_true = (fake_disc_output + 1) / 2
    return prob_fake_true


def disc_cost(gen_weights, disc_weights):
    cost = prob_fake_true(gen_weights, disc_weights) - prob_real_true(disc_weights)
    return cost


def gen_cost(gen_weights, disc_weights):
    return -prob_fake_true(gen_weights, disc_weights)

Training the QGAN
-----------------

We initialize the fixed angles of the "real data" circuit, as well as
the initial parameters for both generator and discriminator. These are
chosen so that the generator initially prepares a state on wire 0 that
is very close to the $\left| 1 \right\rangle$ state.



In [13]:
phi = np.pi / 6
theta = np.pi / 2
omega = np.pi / 7
np.random.seed(0)
eps = 1e-2
init_gen_weights = np.array([np.pi] + [0] * 8) + \
                   np.random.normal(scale=eps, size=(9,))
init_disc_weights = np.random.normal(size=(9,))

We begin by creating the optimizer:



In [14]:
opt = tf.keras.optimizers.SGD(0.4)

Define discriminator optimization procedure:

In [15]:
def disc_optimize(num_epoch, opt, disc_cost, gen_weights, disc_weights, print_progress=True):

    cost = lambda: disc_cost(gen_weights, disc_weights)

    for step in range(num_epoch):
        opt.minimize(cost, disc_weights)
        if print_progress and step % 5 == 0:
            cost_val = cost().numpy()
            print("[disc] Step {}: cost = {}".format(step, cost_val))

Define generator optimization procedure:

In [16]:
def gen_optimize(num_epoch, opt, gen_cost, gen_weights, disc_weights, print_progress=True):

    cost = lambda: gen_cost(gen_weights, disc_weights)

    for step in range(num_epoch):
        opt.minimize(cost, gen_weights)
        if print_progress and step % 5 == 0:
            cost_val = cost().numpy()
            print("[gen]  Step {}: cost = {}".format(step, cost_val))

Define overall optimization strategy with respect to the epoch hyperparameters:

In [17]:
def optimize(num_alternating_epoch, num_individual_epoch, opt,
            disc_cost, init_disc_weights, gen_cost, init_gen_weights,
            print_progress=True):
    gen_weights = tf.Variable(init_gen_weights)
    disc_weights = tf.Variable(init_disc_weights)
    
    for ae in range(num_alternating_epoch):
        if print_progress:
            print('Alternating epoch: ' + str(ae))
        disc_optimize(num_individual_epoch, opt, disc_cost, gen_weights, disc_weights, print_progress=print_progress)
        gen_optimize(num_individual_epoch, opt, gen_cost, gen_weights, disc_weights, print_progress=print_progress)
    
    return gen_weights, disc_weights

Start training:

In [18]:
num_alternating_epoch = 10
num_individual_epoch = 10

gen_weights, disc_weights = optimize(
    num_alternating_epoch, num_individual_epoch, opt,
    disc_cost, init_disc_weights, gen_cost, init_gen_weights,
    print_progress=True)

Alternating epoch: 0
[disc] Step 0: cost = -0.05727687478065491
[disc] Step 5: cost = -0.26348111033439636
[gen]  Step 0: cost = -0.5259977728128433
[gen]  Step 5: cost = -0.7982093915343285
Alternating epoch: 1
[disc] Step 0: cost = 0.003219299018383026
[disc] Step 5: cost = -0.030141867697238922
[gen]  Step 0: cost = -0.8414990827441216
[gen]  Step 5: cost = -0.8914423026144505
Alternating epoch: 2
[disc] Step 0: cost = 0.015962108969688416
[disc] Step 5: cost = 0.004500001668930054
[gen]  Step 0: cost = -0.8957922570407391
[gen]  Step 5: cost = -0.8990886881947517
Alternating epoch: 3
[disc] Step 0: cost = 0.0030284561216831207
[disc] Step 5: cost = 0.001276165246963501
[gen]  Step 0: cost = -0.8971303515136242
[gen]  Step 5: cost = -0.897724699229002
Alternating epoch: 4
[disc] Step 0: cost = 0.000931866466999054
[disc] Step 5: cost = 0.0003459714353084564
[gen]  Step 0: cost = -0.897084329277277
[gen]  Step 5: cost = -0.8973288722336292
Alternating epoch: 5
[disc] Step 0: cost = 0

Check training metrics:

In [20]:
print("Prob(real classified as real): ", prob_real_true(disc_weights).numpy())
print("Prob(fake classified as real): ", prob_fake_true(gen_weights, disc_weights).numpy())
print("Discriminator cost: ", disc_cost(gen_weights, disc_weights).numpy())

Prob(real classified as real):  0.8972049541771412
Prob(fake classified as real):  0.8972050882875919
Discriminator cost:  1.341104507446289e-07


The generator has successfully learned how to simulate the real data
enough to fool the discriminator.

Let's conclude by comparing the states of the real data circuit and the generator. We expect
the generator to have learned to be in a state that is very close to the one prepared in the
real data circuit. An easy way to access the state of the first qubit is through its
`Bloch sphere <https://en.wikipedia.org/wiki/Bloch_sphere>`__ representation:



In [21]:
obs = [qml.PauliX(0), qml.PauliY(0), qml.PauliZ(0)]

bv_real = qml.map(real, obs, dev, interface="tf")
bv_gen = qml.map(generator, obs, dev, interface="tf")

bv_real_arr = bv_real([phi, theta, omega])
bv_gen_arr = bv_gen(gen_weights)
error_norm = np.linalg.norm(bv_real_arr - bv_gen_arr)

print("Real Bloch vector: {}".format(bv_real_arr))
print("Generator Bloch vector: {}".format(bv_gen_arr))
print("Error norm: {}".format(error_norm))

Real Bloch vector: [-0.2169418   0.45048445 -0.86602525]
Generator Bloch vector: [-0.21795285  0.44993043 -0.86605926]
Error norm: 0.0011533907582134387
